In [1]:
import gradio as gr
import pandas as pd
import mlflow

In [17]:
mlflow.set_tracking_uri("http://localhost:5000")

In [9]:
def get_dataframe(feature_zip,feature_squrmeter, feature_rooms, features_ab, feature_wc, feature_ba, feature_bi, feature_frei,
        feature_fh,
        feature_gaeste,
        feature_garage,
        feature_sat,
        feature_lueftung,
        feature_fenster,
        feature_luftwp,
        feature_parkett,
        feature_aufzug,
        feature_reinigung,
        feature_rollstuhl,
        feature_wanne,
        feature_wasch,
        feature_einbaukueche,
        feature_elektro,
        feature_erdwaerme,
        feature_zentral):
    data_list = [feature_zip]+[feature_squrmeter] +[feature_rooms]+ [features_ab] +[feature_wc] + [feature_ba]+[feature_bi]+ [feature_einbaukueche]+ [feature_elektro]+[feature_erdwaerme]+[feature_frei] +[feature_fh]+[feature_gaeste]+[feature_garage]+[feature_sat]+[feature_lueftung]+ [feature_fenster] + [feature_luftwp] + [feature_parkett]+ [feature_aufzug] + [feature_reinigung] + [feature_rollstuhl] + [feature_wanne] +[feature_wasch]+ [feature_zentral]+ ['APARTMENT'] + ['RENT']
    data = pd.DataFrame()
    data = pd.DataFrame(columns=['ZipCode','LivingSpace','Rooms', 'abstellraum', 'bad/wc_getrennt', 'barriefrei', 'bidet', 'einbauk\u00fcche', 'elektro', 'erdwaerme',
                                 'frei', 'fu\u00dfbodenheizung', 'gaestewc', 'garage', 'kable_sat_tv', 'kontrollierte_be-_und_entl\u00fcftungsanlage', 'kunststofffenster',
                                 'luftwp', 'parkett', 'personenaufzug', 'reinigung', 'rollstuhlgerecht', 'wanne', 'wasch_trockenraum','zentralheizung','EstateType_APARTMENT', 'DistributionType_RENT'])
    data.loc[len(data)] = data_list
    print(data)

In [3]:
def load_model(model_name):
    model_name = "rigde-rent-wue"
    model_version = 1
    model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
    return model

In [20]:
x = load_model("ridge-rent-wue")

2023/07/11 15:57:39 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module 'configparser' has no attribute '__version__'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.
2023/07/11 15:57:39 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.8.16`, differs from the version of Python that is currently running, `Python 3.11.3`, and may be incompatible


In [21]:
x

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: e654ce9d65a34446aa4640f6e50a8cbe

In [10]:
with gr.Blocks() as demo:

    gr.Markdown(
    '''
    # Gebe Sie hier an, wie viel qm und wie viele Zimmer ihre Immobilie besitzt 🏠,
    '''
    )
    with gr.Row():
        feature_zip = gr.Textbox(label='Postleitzahl')
    with gr.Row():
        feature_squrmeter = gr.Number(label='Wohnfläche [qm]')
    with gr.Row():
        feature_rooms = gr.Number(label='Anzahl der Zimmer')
    gr.Markdown(
    '''
    # Gebe Sie hier an, welche Voraussetzungen Ihre Immobilie erfüllt 🏠",
    ''')
    
    with gr.Row():
        features_ab = gr.Checkbox(label='Abstellraum')
        feature_wc = gr.Checkbox(label='Bad/WC getrennt')
        feature_ba = gr.Checkbox(label='Barrierefrei')
        feature_bi = gr.Checkbox(label='Bidet')
        feature_frei = gr.Checkbox(label='Frei')
        feature_fh = gr.Checkbox(label='Fußbodenheizung')
        feature_gaeste = gr.Checkbox(label='Gäste-WC')
        feature_garage = gr.Checkbox(label='Garage')
        feature_sat = gr.Checkbox(label='Kabel/Sat-TV')
        feature_lueftung = gr.Checkbox(label='Kontrollierte Be- und Entlüftungsanlage')
        feature_fenster = gr.Checkbox(label='Kunststofffenster')
        feature_luftwp = gr.Checkbox(label='Luft-WP')
        feature_parkett = gr.Checkbox(label='Parkett')
        feature_aufzug = gr.Checkbox(label='Personenaufzug')
        feature_reinigung = gr.Checkbox(label='Reinigung')
        feature_rollstuhl = gr.Checkbox(label='Rollstuhlgerecht')
        feature_wanne = gr.Checkbox(label='Wanne')
        feature_wasch = gr.Checkbox(label='Wasch-/Trockenraum')
        feature_einbaukueche = gr.Checkbox(label='Einbauküche')
        feature_elektro = gr.Checkbox(label='Elektro')
        feature_erdwaerme = gr.Checkbox(label='Erdwärme')
        feature_zentral = gr.Checkbox(label='Zentralheizung')
    btn = gr.Button('hier')
    btn.click(fn=get_dataframe, inputs=[feature_zip,feature_squrmeter, feature_rooms, features_ab, feature_wc, feature_ba, feature_bi, feature_frei,
            feature_fh,
            feature_gaeste,
            feature_garage,
            feature_sat,
            feature_lueftung,
            feature_fenster,
            feature_luftwp,
            feature_parkett,
            feature_aufzug,
            feature_reinigung,
            feature_rollstuhl,
            feature_wanne,
            feature_wasch,
            feature_einbaukueche,
            feature_elektro,
            feature_erdwaerme,
            feature_zentral])
demo.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


   ZipCode  LivingSpace  Rooms  abstellraum  bad/wc_getrennt  barriefrei  \
0  5589655          0.0    0.0        False            False       False   

   bidet  einbauküche  elektro  erdwaerme  ...  luftwp  parkett  \
0  False        False    False      False  ...   False    False   

   personenaufzug  reinigung  rollstuhlgerecht  wanne  wasch_trockenraum  \
0           False      False             False  False              False   

   zentralheizung  EstateType_APARTMENT  DistributionType_RENT  
0           False             APARTMENT                   RENT  

[1 rows x 27 columns]
